In [ ]:
from tools import *
from src.experiment.file_analysis import *
import traceback

## Load data

In [ ]:

exp_name_list=[]
for file in RESULT_DIR_PATH.iterdir():
    if file.is_dir():
        exp_name_list.append(file.name)
        print(file.name)
all_data_dict={}
all_hcbo_info_dict={}
def analyze_exp_name(exp_name):
    ret_name=""
    if(exp_name=="CoralGraph"):
        ret_name="Coral Ecology"
        D=11
        I=5
    elif(exp_name=="ProteinGraph"):
        ret_name="Protein"
        D=5
        I=3
    elif(exp_name=="HealthGraph"):
        ret_name="Health"
        D=10
        I=3
    else:
        last_dash_index =exp_name.rfind("-")  
        if last_dash_index != -1:  
            second_last_dash_index = exp_name.rfind("-", 0, last_dash_index) 
            if second_last_dash_index != -1:
                real_name = exp_name[:second_last_dash_index]
                D_str = exp_name[second_last_dash_index+1:last_dash_index]
        ret_name=real_name
        D=int(D_str)
        I=D//3
    return ret_name,D,I

for exp_name in exp_name_list:
    if(exp_name in ["HealthGraph"]):
        maximize=False
    else:
        maximize=True
    if exp_name in ["EffDim"]:
        continue
    try:
        result_reader=ResultManager(exp_name)
        convergence_data_dict={}
        data_dict=result_reader.read_hdo_baseline_results()
        convergence_data_dict=analyze_hdo_results(data_dict,maximize)
        hcbo_result_dict={}
        for name in ["HCBO","CBO","Random-Search"]:
            try:            
                hcbo_result_dict[name]=result_reader.read_causal_baseline_result(name)
            except Exception as e:
                print(e)
        hcbo_covergence_data_dict,hcbo_report_info_dict,issf_info_list_dict=analyze_hcbo_results(
            hcbo_result_dict
            ,maximize=maximize) 
        convergence_data_dict={**convergence_data_dict,**hcbo_covergence_data_dict}
        exp_identifier=exp_name
        all_data_dict[exp_identifier]=convergence_data_dict
        all_hcbo_info_dict[exp_identifier]=hcbo_report_info_dict
    except Exception as e:
        print("Fail in ",exp_name)
        print(e)
        traceback.print_exc()

## Draw the figure without normalization

In [ ]:
fig=draw_convergence_line_graph(
    all_data_dict,
    "Cumulative Cost",
    "$Y^*$",subplot_distance_dict={
        "wspace":0.15
    }
)
fig.show()

## Statistical tests on baseline experiment results

In [ ]:
import scipy.stats as stats
for key,value_dict in all_data_dict.items():
    print(f"---{key}---")
    baseline_sample={}
    HCBO_result=None
    for baseline_name,value in value_dict.items():
        if(baseline_name=="HCBO"):
            if(HCBO_result is not None):
                raise ValueError("Illegal Input")
            HCBO_result=value[:,-1]
        else:
            baseline_sample[baseline_name]=value[:,-1]
    for baseline_name,comp_result in baseline_sample.items():
        if(key=="HealthGraph"):
            alternative="less"
        else:
            alternative="greater"
        t_stat, p_value = stats.ttest_rel(HCBO_result, comp_result,alternative=alternative)
        print(f"{baseline_name:<15} t Value: {t_stat:<10}, p Value: {p_value:<10}")
